# import library & define functions

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy

/Users/kaikeda/.pyenv/versions/3.6.8/envs/python3/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/kaikeda/.pyenv/versions/3.6.8/envs/python3/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [1]:
class Sudoku:
    def __init__(self, initial_data):
        self.result = initial_data
        self.tmp =  np.repeat(np.ones((9, 9))[:, :, np.newaxis], 9, axis=2)
        self.block_exist_place()
        self.block_all()
        print('imported data:', 'null size:', self.result_zero_size())
        self.display_result()
        self.recursion_cnt = 0
        self.exploring_num = 0
        
    def result_zero_size(self):
        return np.count_nonzero(self.result.flatten() == 0)
    
    def block_exist_place(self):
        non_zero_index = pd.DataFrame(np.where(self.result != 0))
        for i in list(non_zero_index.transpose().index):
            self.tmp[:, non_zero_index[i][0], non_zero_index[i][1]] = 0    

    def put_in_result(self, r, c, v):
        self.result[r, c] = v
        self.block_exist_place()
        self.block_all()
#         self.display_by_color(pd.DataFrame(self.result), pd.IndexSlice[r:r, c:c])
    
    def block_all(self):
        for i in list(np.arange(9)):
            # row
            for v in self.result[i][self.result[i] != 0]:
                self.tmp[v-1,i,:] = 0
            # column
            for v in self.result[:,i][self.result[:,i] != 0]:
                self.tmp[v-1,:,i] = 0
        # box
        for a in list(np.arange(0, 7, 3)):
            for b in list(np.arange(0, 7, 3)):
                t_box_arr = self.result[a:a+3, b:b+3].flatten()
                for v in t_box_arr[t_box_arr != 0]:
                    self.tmp[v-1,a:a+3, b:b+3] = 0

    def put_all(self):
        for n in list(np.arange(9)):
            num_tmp = self.tmp[n]

            for i in list(np.arange(9)):
                # row
                if num_tmp[i].sum() == 1:
                    self.put_in_result(i, np.where(num_tmp[i] == 1)[0][0], n+1)
                # column
                if num_tmp[:,i].sum() == 1:
                    self.put_in_result(np.where(num_tmp[:,i] == 1)[0][0], i, n+1)
            # box
            for a in list(np.arange(0, 7, 3)):
                for b in list(np.arange(0, 7, 3)):
                    t_box = num_tmp[a:a+3, b:b+3]
                    if t_box.flatten().sum() == 1:
                        self.put_in_result(a + np.where(t_box == 1)[0][0], b + np.where(t_box == 1)[1][0], n+1)

    def block_and_put(self):
        while self.result_zero_size() != 0:
            before_size = self.result_zero_size()
            print('.')
            self.put_all()
            if self.result_zero_size() == before_size:
                break

    def exploratory_calc(self):
        candidate_size_map = np.sum(self.tmp, axis=0)
        min_size = candidate_size_map[candidate_size_map != 0].min()
        min_size_indexes = pd.DataFrame(np.where(candidate_size_map == min_size))
        for i in list(min_size_indexes.transpose().index):
            candidate_values = list(np.where(self.tmp[:, min_size_indexes[i][0], min_size_indexes[i][1]] == 1)[0] + 1)
            for v in candidate_values:
                self.put_in_result(min_size_indexes[i][0], min_size_indexes[i][1], v)
                self.block_and_put()
                
                self.check_result_duplicate()
                
                if self.result_zero_size() == 0:
                    return True
                if np.count_nonzero(self.tmp.flatten() != 0) != 0:
                    # NOTE: append savepoint
                    self.recursion_cnt += 1
                    print('recursion_cnt: ', self.recursion_cnt)
                    self.result_bk.append(deepcopy(self.result))
                    self.tmp_bk.append(deepcopy(self.tmp))
                    # NOTE: recurrent run
                    if self.exploratory_calc() == True:
                        return True
                # NOTE: rollback
                self.result = deepcopy(self.result_bk[self.recursion_cnt])
                self.tmp = deepcopy(self.tmp_bk[self.recursion_cnt])
        self.recursion_cnt -= 1
        self.result_bk = self.result_bk[:-1]
        self.tmp_bk = self.tmp_bk[:-1]
        return False  
        
    def calc(self):
        self.block_and_put()
        self.display_result()
        if self.result_zero_size() == 0:
            self.check_result()
            print('complete!')
        else:
#             print('unresolved size:', self.result_zero_size())
            self.result_bk = [deepcopy(self.result)]
            self.tmp_bk = [deepcopy(self.tmp)]
            self.total_candidate_size = pd.DataFrame(np.where(self.tmp != 0)).transpose().index.size
            print('total candidate size: ', self.total_candidate_size)
#             self.exploratory_calc()
            if self.result_zero_size() == 0:
                self.check_result()
                print('complete!')
            else:
                print('unresolved size:', self.result_zero_size())
                self.display_result()

                
    # debug functions:
    
    def display_result(self):
        display(pd.DataFrame(self.result))
    
    def display_remaining_candidate(self):
        for i in np.arange(9):
            candidate_size = np.count_nonzero(self.tmp[i].flatten() != 0)
            if candidate_size != 0:
                print('num:', i+1)
                display(pd.DataFrame(self.tmp[i].astype(int)))

    def raise_err(self):
        raise Exception('anything is wrong!')

    def check_result(self):
        for i in list(np.arange(9)):
            if np.unique(self.result[i]).size != 9:
                display( pd.DataFrame(self.result).style.background_gradient(cmap='winter', subset=pd.IndexSlice[i:i]) )
                self.raise_err()
            if np.unique(self.result[:,i]).size != 9:
                display( pd.DataFrame(self.result).style.background_gradient(cmap='winter', subset=[i]) )
                self.raise_err()
        for a in list(np.arange(0, 7, 3)):
            for b in list(np.arange(0, 7, 3)):
                if np.unique(self.result[a:a+3, b:b+3].flatten()).size != 9:
                    display( pd.DataFrame(self.result).style.background_gradient(cmap='winter', subset=pd.IndexSlice[a:a+2, b:b+2]) )
                    self.raise_err()

    def check_result_duplicate(self):
        for i in list(np.arange(9)):
            if np.unique(self.result[i][self.result[i] != 0]).size != self.result[i][self.result[i] != 0].size:
                display( pd.DataFrame(self.result).style.background_gradient(cmap='winter', subset=pd.IndexSlice[i:i]) )
                self.raise_err()
            if np.unique(self.result[:,i][self.result[:,i] != 0]).size != self.result[:,i][self.result[:,i] != 0].size:
                display( pd.DataFrame(self.result).style.background_gradient(cmap='winter', subset=[i]) )
                self.raise_err()
        for a in list(np.arange(0, 7, 3)):
            for b in list(np.arange(0, 7, 3)):
                t = self.result[a:a+3, b:b+3].flatten()
                if (np.unique(t[t != 0]).size) != (t[t != 0].size):
                    display( pd.DataFrame(self.result).style.background_gradient(cmap='winter', subset=pd.IndexSlice[a:a+2, b:b+2]) )
                    self.raise_err()

    def display_by_color(self, df, subset):
        display( df.style.background_gradient(cmap='winter', subset=subset) )